In [54]:
# install

In [1]:
# pip install --upgrade pyspark

In [2]:
import pyspark
print(pyspark.__version__)

3.5.2


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# initializes a SparkSession. To interact with Spark's functionality.

In [5]:
spark = SparkSession.builder.appName("Data Analysis").getOrCreate()

# Read the CSVs Files

In [6]:
bridge_cust_loan_df=  spark.read.csv('bridge_cust_loan.csv', header=True, inferSchema=True)

In [7]:
customers_df=spark.read.csv('customers.csv', header=True, inferSchema=True)

In [8]:
loan_type_df= spark.read.csv('loan_type.csv', header=True, inferSchema=True)

In [9]:
time_dim_df=spark.read.csv('time_dim.csv', header=True, inferSchema=True)

In [10]:
customer_financials_df= spark.read.csv('customer_financials.csv', header=True, inferSchema=True)

# simple show for data basic info

In [11]:
customers_df.show(10)

+------------+--------------------+
|Loan_Type_ID|        Type_of_Loan|
+------------+--------------------+
|           1|           Auto Loan|
|           2| Credit-Builder Loan|
|           3|       Personal Loan|
|           4|    Home Equity Loan|
|           5|       Not Specified|
|           6|       Not Specified|
|           7|       Mortgage Loan|
|           8|        Student Loan|
|           9|Debt Consolidatio...|
|          10|           Auto Loan|
+------------+--------------------+
only showing top 10 rows



In [12]:
customers_df.printSchema()
loan_type_df.printSchema()
customer_financials_df.printSchema()
time_dim_df.printSchema()
bridge_cust_loan_df.printSchema()

root
 |-- Loan_Type_ID: integer (nullable = true)
 |-- Type_of_Loan: string (nullable = true)

root
 |-- Loan_Type_ID: integer (nullable = true)
 |-- Type_of_Loan: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Month_ID: integer (nullable = true)
 |-- Annual_Income: double (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: double (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Total_EMI_per_month: double (nullable = true)
 |-- Amount_invested_monthly: double (nullable = true)
 |-- Monthly_Balance: double (nu

In [13]:
print("Number of customers:", customers_df.count())
print("Number of loans:", loan_type_df.count())

# Get basic statistics for numerical columns
customers_df.describe().show()
loan_type_df.describe().show()

Number of customers: 18
Number of loans: 18
+-------+-----------------+------------+
|summary|     Loan_Type_ID|Type_of_Loan|
+-------+-----------------+------------+
|  count|               18|          18|
|   mean|              9.5|        NULL|
| stddev|5.338539126015656|        NULL|
|    min|                1|   Auto Loan|
|    max|               18|Student Loan|
+-------+-----------------+------------+

+-------+-----------------+------------+
|summary|     Loan_Type_ID|Type_of_Loan|
+-------+-----------------+------------+
|  count|               18|          18|
|   mean|              9.5|        NULL|
| stddev|5.338539126015656|        NULL|
|    min|                1|   Auto Loan|
|    max|               18|Student Loan|
+-------+-----------------+------------+



In [14]:
customers_df.select("Loan_Type_ID","Type_of_Loan")

DataFrame[Loan_Type_ID: int, Type_of_Loan: string]

In [15]:
print(customers_df.columns)
print(loan_type_df.columns)

['Loan_Type_ID', 'Type_of_Loan']
['Loan_Type_ID', 'Type_of_Loan']


# Select all loan types:


In [16]:
loan_types = loan_type_df.select("*")
loan_types.show()

+------------+--------------------+
|Loan_Type_ID|        Type_of_Loan|
+------------+--------------------+
|           1|           Auto Loan|
|           2| Credit-Builder Loan|
|           3|       Personal Loan|
|           4|    Home Equity Loan|
|           5|       Not Specified|
|           6|       Not Specified|
|           7|       Mortgage Loan|
|           8|        Student Loan|
|           9|Debt Consolidatio...|
|          10|           Auto Loan|
|          11|         Payday Loan|
|          12|         Payday Loan|
|          13|        Student Loan|
|          14|       Personal Loan|
|          15|    Home Equity Loan|
|          16|       Mortgage Loan|
|          17|Debt Consolidatio...|
|          18| Credit-Builder Loan|
+------------+--------------------+



# Count the number of loan types:

In [17]:
loan_type_count = loan_type_df.count()
print(loan_type_count)

18


# Find the average annual income of customers:

In [18]:
avg_annual_income = customer_financials_df.agg({"Annual_Income": "avg"}).collect()[0][0]
print(f"Average Annual Income: ${avg_annual_income:,.2f}")

Average Annual Income: $176,415.70


# Find the total outstanding debt for each customer:


In [21]:
total_debt_per_customer = customer_financials_df.groupBy("Customer_ID").agg({"Outstanding_Debt": "sum"})
total_debt_per_customer.show()

+-----------+---------------------+
|Customer_ID|sum(Outstanding_Debt)|
+-----------+---------------------+
|       8638|    63075.59999999995|
|      26623|             298214.4|
|       8086|    5293.440000000001|
|      40653|   15527.040000000014|
|       4900|   108120.32000000008|
|      39161|   100794.24000000015|
|      25227|   29512.320000000014|
|       2811|    88500.48000000011|
|      26936|    55160.56000000002|
|      19758|    95795.20000000011|
|      22970|     67093.7599999999|
|      19669|    46773.75999999994|
|      12006|    60421.19999999994|
|       7387|    60407.04000000003|
|      50021|    92567.67999999996|
|       8257|    9934.079999999996|
|      30617|             67211.52|
|      12393|    50924.16000000003|
|      50320|    9777.280000000015|
|      14324|   124728.32000000011|
+-----------+---------------------+
only showing top 20 rows



# Find customers with an annual income greater than $50,000:

In [23]:
high_income_customers = customer_financials_df.filter(customer_financials_df.Annual_Income > 50000) \
                                              .select("Customer_ID", "Annual_Income", "Monthly_Inhand_Salary", "Credit_Score")
high_income_customers.show(truncate=False)

+-----------+-------------+---------------------+------------+
|Customer_ID|Annual_Income|Monthly_Inhand_Salary|Credit_Score|
+-----------+-------------+---------------------+------------+
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good        |
|11708      |143162.64    |12187.22             |Good  

#  Find the total outstanding debt for each customer


In [24]:
total_debt_per_customer = customer_financials_df.groupBy("Customer_ID").agg({"Outstanding_Debt": "sum"})
display(total_debt_per_customer.toPandas())

,Customer_ID,sum(Outstanding_Debt)
0,8638,63075.60
1,26623,298214.40
2,8086,5293.44
3,40653,15527.04
4,4900,108120.32
...,...,...
12495,36480,163749.12
12496,16596,30599.04
12497,29854,125550.72
12498,7874,281906.56


# Filter customers who have delayed payment more than 5 times:

In [25]:
delayed_customers = customer_financials_df.filter(customer_financials_df.Num_of_Delayed_Payment > 5)
display(delayed_customers.select("Customer_ID", "Num_of_Delayed_Payment", "Credit_Score").toPandas())

,Customer_ID,Num_of_Delayed_Payment,Credit_Score
0,3392,7.0,Good
1,3392,7.0,Good
2,3392,7.0,Good
3,3392,7.0,Good
4,3392,7.0,Good
...,...,...,...
710299,37932,6.0,Poor
710300,37932,6.0,Poor
710301,37932,6.0,Poor
710302,37932,6.0,Poor


# Join customer_financials_df with loan_type_df to get Loan Type

In [27]:
# Join the customer_financials_df with the bridge_cust_loan_df to get the Loan_Type_ID
loan_credit_data = customer_financials_df.join(bridge_cust_loan_df, "Customer_ID", "inner") \
                                         .join(loan_type_df, "Loan_Type_ID", "inner") \
                                         .select("Type_of_Loan", "Credit_Score", "Num_of_Loan")

loan_credit_data.show()

+-------------------+------------+-----------+
|       Type_of_Loan|Credit_Score|Num_of_Loan|
+-------------------+------------+-----------+
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Person

# Filter Customers with Poor Credit Score

In [29]:
poor_credit_customers = loan_credit_joined.filter(loan_credit_joined.Credit_Score == "Poor")
poor_credit_customers.show()

+-------------+------------+-----------+
| Type_of_Loan|Credit_Score|Num_of_Loan|
+-------------+------------+-----------+
| Student Loan|        Poor|          3|
| Student Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|Not Specified|        Poor|          3|
|Not Specified|        Poor|          3|
| Student Loan|        Poor|          3|
| Student Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|Not Specified|        Poor|          3|
|Not Specified|        Poor|          3|
| Student Loan|        Poor|          3|
| Student Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|    Auto Loan|        Poor|          3|
|Not Specified|        Poor|          3|
|Not Specified|        Poor|          3|
| Student Loan|        Poor|          3|
| Student Loan|        Poor|          3|
+-------------+------------+-----------+
only showing top

# Count Customers by Loan Type and Credit Score

In [30]:
loan_credit_grouped = loan_credit_joined.groupBy("Type_of_Loan", "Credit_Score").count()
loan_credit_grouped.show()

+--------------------+------------+-------+
|        Type_of_Loan|Credit_Score|  count|
+--------------------+------------+-------+
|Debt Consolidatio...|        Good| 529280|
|       Not Specified|        Poor|2135296|
|    Home Equity Loan|        Good| 556544|
|           Auto Loan|        Poor|1916160|
|       Mortgage Loan|        Good| 558720|
|       Mortgage Loan|        Poor|1947264|
|       Personal Loan|        Good| 533120|
|        Student Loan|        Poor|1970176|
|       Personal Loan|    Standard|2524160|
| Credit-Builder Loan|    Standard|2589696|
|    Home Equity Loan|    Standard|2509056|
|       Mortgage Loan|    Standard|2477824|
| Credit-Builder Loan|        Poor|2005120|
|Debt Consolidatio...|        Poor|1951232|
|        Student Loan|    Standard|2438784|
|         Payday Loan|    Standard|2581760|
|       Not Specified|    Standard|3364352|
|           Auto Loan|        Good| 523392|
|Debt Consolidatio...|    Standard|2482816|
|         Payday Loan|        Go

# Find Average Number of Loans per Customer Based on Loan Type and Credit Score

In [31]:
avg_num_loans = loan_credit_joined.groupBy("Type_of_Loan", "Credit_Score") \
                                  .agg({"Num_of_Loan": "avg"})
avg_num_loans.show()

+--------------------+------------+------------------+
|        Type_of_Loan|Credit_Score|  avg(Num_of_Loan)|
+--------------------+------------+------------------+
|Debt Consolidatio...|        Good|3.9901639344262296|
|       Not Specified|        Poor|  4.70413940613991|
|    Home Equity Loan|        Good| 2.227305562244161|
|           Auto Loan|        Poor| 4.625508914783097|
|       Mortgage Loan|        Good| 5.600432484382509|
|       Mortgage Loan|        Poor| 4.493277252982142|
|       Personal Loan|        Good|2.6077097505668934|
|        Student Loan|        Poor| 5.073265836639759|
|       Personal Loan|    Standard| 3.213100762788713|
| Credit-Builder Loan|    Standard| 4.240213892638355|
|    Home Equity Loan|    Standard|  3.76654362595624|
|       Mortgage Loan|    Standard| 4.583224896985469|
| Credit-Builder Loan|        Poor| 4.529675777063237|
|Debt Consolidatio...|        Poor| 4.753281561404714|
|        Student Loan|    Standard|3.6352733101199357|
|         

# Select Customers with More than 3 Loans

In [32]:
many_loans_customers = loan_credit_joined.filter(loan_credit_joined.Num_of_Loan > 3)
many_loans_customers.show()

+-------------------+------------+-----------+
|       Type_of_Loan|Credit_Score|Num_of_Loan|
+-------------------+------------+-----------+
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Person

# Select Unique Loan Types

In [33]:
unique_loan_types = loan_credit_joined.select("Type_of_Loan").distinct()
unique_loan_types.show()

+--------------------+
|        Type_of_Loan|
+--------------------+
|    Home Equity Loan|
|         Payday Loan|
|       Personal Loan|
|Debt Consolidatio...|
|       Mortgage Loan|
|        Student Loan|
| Credit-Builder Loan|
|           Auto Loan|
|       Not Specified|
+--------------------+



# Calculate Total Number of Loans for Each Loan Type

In [34]:
total_loans_by_type = loan_credit_joined.groupBy("Type_of_Loan") \
                                        .agg({"Num_of_Loan": "sum"})
total_loans_by_type.show()

+--------------------+----------------+
|        Type_of_Loan|sum(Num_of_Loan)|
+--------------------+----------------+
|    Home Equity Loan|     1.8938624E7|
|         Payday Loan|     2.0531328E7|
|       Personal Loan|     1.9558528E7|
|Debt Consolidatio...|     1.9892608E7|
|       Mortgage Loan|     2.2145024E7|
|        Student Loan|     1.9220992E7|
| Credit-Builder Loan|     2.0587264E7|
|           Auto Loan|     2.0604032E7|
|       Not Specified|     2.2457216E7|
+--------------------+----------------+



# Sort Customers by Number of Loans in Descending Order

In [35]:
sorted_loans_customers = loan_credit_joined.orderBy(loan_credit_joined.Num_of_Loan.desc())
sorted_loans_customers.show()

+--------------------+------------+-----------+
|        Type_of_Loan|Credit_Score|Num_of_Loan|
+--------------------+------------+-----------+
|        Student Loan|    Standard|         9_|
|         Payday Loan|    Standard|         9_|
|         Payday Loan|        Poor|         9_|
|Debt Consolidatio...|    Standard|         9_|
|Debt Consolidatio...|        Poor|         9_|
|         Payday Loan|    Standard|         9_|
| Credit-Builder Loan|        Poor|         9_|
|Debt Consolidatio...|    Standard|         9_|
|       Personal Loan|        Poor|         9_|
|Debt Consolidatio...|    Standard|         9_|
|         Payday Loan|        Poor|         9_|
|    Home Equity Loan|    Standard|         9_|
|       Personal Loan|        Poor|         9_|
|    Home Equity Loan|    Standard|         9_|
| Credit-Builder Loan|        Poor|         9_|
|       Mortgage Loan|    Standard|         9_|
|       Personal Loan|    Standard|         9_|
|       Mortgage Loan|    Standard|     

# Count Number of Customers in Each Credit Score Category

In [36]:
credit_score_counts = loan_credit_joined.groupBy("Credit_Score").count()
credit_score_counts.show()

+------------+--------+
|Credit_Score|   count|
+------------+--------+
|        Good| 5466112|
|        Poor|17823104|
|    Standard|23391872|
+------------+--------+



# Select Customers with a Specific Loan Type 'Mortgage'

In [37]:
mortgage_customers = loan_credit_joined.filter(loan_credit_joined.Type_of_Loan == "Mortgage")
mortgage_customers.show()

+------------+------------+-----------+
|Type_of_Loan|Credit_Score|Num_of_Loan|
+------------+------------+-----------+
+------------+------------+-----------+



#  Identify Customers with the Highest Number of Loans

In [38]:
max_loans = loan_credit_joined.agg({"Num_of_Loan": "max"})
max_loans.show()


+----------------+
|max(Num_of_Loan)|
+----------------+
|              9_|
+----------------+



# Filter Out Customers with Null Values in Num_of_Loan 

In [39]:
non_null_loans = loan_credit_joined.filter(loan_credit_joined.Num_of_Loan.isNotNull())
non_null_loans.show()

+-------------------+------------+-----------+
|       Type_of_Loan|Credit_Score|Num_of_Loan|
+-------------------+------------+-----------+
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Person

# Calculate the Average Credit Score per Loan Type

In [40]:
avg_credit_score = loan_credit_joined.groupBy("Type_of_Loan") \
                                     .agg({"Credit_Score": "avg"})
avg_credit_score.show()

+--------------------+-----------------+
|        Type_of_Loan|avg(Credit_Score)|
+--------------------+-----------------+
|    Home Equity Loan|             NULL|
|         Payday Loan|             NULL|
|       Personal Loan|             NULL|
|Debt Consolidatio...|             NULL|
|       Mortgage Loan|             NULL|
|        Student Loan|             NULL|
| Credit-Builder Loan|             NULL|
|           Auto Loan|             NULL|
|       Not Specified|             NULL|
+--------------------+-----------------+



# Select Only Customers with Good or Excellent Credit Scores


In [41]:
good_credit_customers = loan_credit_joined.filter(loan_credit_joined.Credit_Score.isin("Good", "Excellent"))
good_credit_customers.show()

+-------------------+------------+-----------+
|       Type_of_Loan|Credit_Score|Num_of_Loan|
+-------------------+------------+-----------+
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|      Personal Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|Credit-Builder Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|          Auto Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|   Home Equity Loan|        Good|          4|
|      Person

#  Count the Number of Distinct Loan Types

In [42]:
distinct_loan_type_count = loan_credit_joined.select("Type_of_Loan").distinct().count()
print(f"Number of distinct loan types: {distinct_loan_type_count}")

Number of distinct loan types: 9


# Select Customers Who Have Delayed Payments and Have a Poor Credit Score

In [43]:
delayed_poor_credit_customers = customer_financials_df.filter(
    (customer_financials_df.Num_of_Delayed_Payment > 0) & 
    (customer_financials_df.Credit_Score == "Poor")
).select("Customer_ID", "Credit_Score", "Num_of_Loan")
delayed_poor_credit_customers.show()

+-----------+------------+-----------+
|Customer_ID|Credit_Score|Num_of_Loan|
+-----------+------------+-----------+
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
|      21511|        Poor|          3|
+-----------+------------+-----------+
only showing top 20 rows



# Identify Loan Types with the Most Customers

In [44]:
popular_loan_types = loan_credit_joined.groupBy("Type_of_Loan") \
                                       .count() \
                                       .orderBy("count", ascending=False)
popular_loan_types.show()

+--------------------+-------+
|        Type_of_Loan|  count|
+--------------------+-------+
|       Not Specified|6531072|
|         Payday Loan|5192704|
| Credit-Builder Loan|5176320|
|    Home Equity Loan|5005312|
|        Student Loan|4987904|
|       Mortgage Loan|4983808|
|       Personal Loan|4977664|
|Debt Consolidatio...|4963328|
|           Auto Loan|4862976|
+--------------------+-------+



# Calculate Sum of Loans for Each Credit Score Category

In [45]:
sum_loans_credit_score = loan_credit_joined.groupBy("Credit_Score") \
                                           .agg({"Num_of_Loan": "sum"})
sum_loans_credit_score.show()

+------------+----------------+
|Credit_Score|sum(Num_of_Loan)|
+------------+----------------+
|        Good|     1.5771648E7|
|        Poor|     8.1755904E7|
|    Standard|     8.6408064E7|
+------------+----------------+

